In [2]:
import random
import csv

# S-Box: Used for substitution operation, providing a non-linear substitution step.
S_Box = [0xE, 0x4, 0xD, 0x1, 0x2, 0xF, 0xB, 0x8, 0x3, 0xA, 0x6, 0xC, 0x5, 0x9, 0x0, 0x7]

# P-Box: Used for permutation operation, it rearranges the input bits.
P_Box = [1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15, 4, 8, 12, 16]

# Generate five 16-bit subkeys from a 32-bit master key.
def gen_K_list(K):
    Ks = []
    for _ in range(5):
        Ks.append(K & 0xFFFF)  # Get the last 16 bits of K
        K >>= 4  # Right shift by 4 bits
    return Ks[::-1]

# Perform substitution operation using the S-Box.
def pi_s(s_box, ur):
    vr = 0
    for i in range(4):
        uri = ur & 0xF  # Get the last 4 bits of ur
        vri = s_box[uri]  # Get the substitute value from the S-Box
        vr += vri << (4 * i)  # Store the result in vr
        ur >>= 4  # Right shift by 4 bits
    return vr

# Perform permutation operation using the P-Box.
def pi_p(p_box, vr):
    wr = 0
    for i in range(15, -1, -1):
        vri = vr & 1  # Get the last bit of vr
        vr >>= 1  # Right shift by 1 bit
        wr |= vri << (16 - p_box[i])  # Rearrange bits based on P-Box value
    return wr

# Implement the SPN encryption operation.
def do_SPN(x, s_box, p_box, Ks):
    wr = x
    # Execute Nr-1 rounds of encryption, each round includes three steps: Key addition, substitution, and permutation.
    for r in range(3):
        ur = wr ^ Ks[r]  # Key addition
        vr = pi_s(s_box, ur)  # Substitution
        wr = pi_p(p_box, vr)  # Permutation
    # The last round does not include permutation.
    ur = wr ^ Ks[3]  # Key addition
    vr = pi_s(s_box, ur)  # Substitution
    return vr ^ Ks[4]  # Output the result after another key addition

# Encrypt a 16-bit plaintext with a given 32-bit key.
def encrypt(K, x):
    Ks = gen_K_list(K)  # Generate subkeys
    return do_SPN(x, S_Box, P_Box, Ks)  # Execute SPN encryption

# Generate plaintext-ciphertext pairs
def generate_pairs():
    # 0011 1010 1001 0100 1101 0110 0011 1111
    K = int("00111010100101001101011000111111", 2)
    
    pairs = []
    # 设置明文-密文对个数
    for _ in range(10000):
        plaintext = random.randint(0, 0xFFFF)
        ciphertext = encrypt(K, plaintext)
        pairs.append((format(plaintext, '016b'), format(ciphertext, '016b')))
    
    return pairs

# Save the pairs to a CSV file
def save_to_csv(pairs, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['Plaintext', 'Ciphertext']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for p, c in pairs:
            writer.writerow({'Plaintext': p, 'Ciphertext': c})

# Generate and save plaintext-ciphertext pairs to a CSV file
pairs = generate_pairs()
filename = "./data/明文_密文对.csv"
save_to_csv(pairs, filename)

In [3]:
import pandas as pd

# 读取CSV文件
df1 = pd.read_csv("./data/明文_密文对.csv")
df2 = pd.read_csv("./data/Y与K异或结果.csv")
# 显示前五行
df1.head(5)


,Plaintext,Ciphertext
0,110110010111101,101010110001000
1,110101010000111,1101010000001100
2,1110111101100111,100111010111000
3,1100100011111100,1100011110110011
4,1011100011011100,1001010010110011


In [4]:
df2.head(5)

,U5,U6,U7,U8,U13,U14,U15,U16
0,1,0,1,0,0,1,1,1
1,1,0,1,1,0,0,1,1
2,0,0,0,1,0,1,1,1
3,1,0,0,0,1,1,0,0
4,1,0,1,1,1,1,0,0


In [5]:
df3 = pd.read_csv("./data/S盒逆置换结果.csv")
df3.head(5)

,u5,u6,u7,u8,u13,u14,u15,u16
0,1,0,0,1,1,1,1,1
1,0,1,1,0,1,0,0,0
2,0,0,1,1,1,1,1,1
3,0,1,1,1,1,0,1,1
4,0,1,1,0,1,0,1,1


In [6]:
df4 = pd.read_csv("./data/偏差统计结果.csv")
df4.head(5)

,Keys,Bias
0,0,0.0071
1,1,0.0056
2,10,0.0000
3,11,0.0009
4,100,0.0017


In [7]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv("./data/偏差统计结果.csv")

# 打印前五行
for index, row in df.head().iterrows():
    # 按格式输出
    print(f"{row['Keys']}, {row['Bias']:.6f}")


0.0, 0.007100
1.0, 0.005600
10.0, 0.000000
11.0, 0.000900
100.0, 0.001700
